In [2]:
import cv2 as cv
import os
import torch
import numpy as np
import time

version = "21"
# 读入图像路径
train_image_path = r"..\robotdetection-"+version+r"\train\images"
train_label_path = r"..\robotdetection-"+version+r"\train\labels"
valid_image_path = r"..\robotdetection-"+version+r"\valid\images"
valid_label_path = r"..\robotdetection-"+version+r"\valid\labels"

train_files = os.listdir(train_image_path)
train_size = len(train_files)
valid_files = os.listdir(valid_image_path)
valid_size = len(valid_files)

model = torch.load("model_csp3_800.pth")
count = 0

start = time.time()
for i, file in enumerate(train_files):
    image = cv.imread(train_image_path + "\\" + file)
    color_image = image
    # print(color_image.shape)
    image_norm = cv.normalize(image, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    image_t = torch.from_numpy(np.array(image_norm, dtype=np.float32)).permute(2, 0, 1).unsqueeze(0)
    pred = model(image_t.to('cuda')).squeeze().to('cpu').float()
    count += 1
end  = time.time()

print("inference time:{:.2f} ms".format((end-start)/count*1000))

inference time:8.49 ms


In [7]:
%cd ../yolov5

from pathlib import Path
import time
# 从摄像头中读入图片
import numpy as np
import torch
from ultralytics.utils.plotting import Annotator
from  ultralytics.utils.plotting import colors
from models.common import DetectMultiBackend
from utils.augmentations import letterbox
from utils.general import check_img_size
from utils.general import non_max_suppression
from utils.torch_utils import select_device
from  utils.general import scale_boxes
import pyrealsense2 as rs
import cv2 as cv
import time
import  os

# 打开摄像头，0表示使用默认的摄像头
# 导入模型

ROOT = r"C:\Users\29777\Desktop\paper\object_detection\code\car_follow_diy\yolov5" # yolopractice/YoloComputerCameraObjectDetection.py
device = 0
device = select_device(device)
weights = ROOT + '/last.pt'
dnn = False
imgsz = (640, 640)
data = ROOT + '/data/coco128.yaml',  # dataset.yaml path
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=False)
# torch.save(model,"last_save.pt")
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # check image size
# 创建一个VideoWriter对象
version = "17"
# 读入图像路径
train_image_path = r"..\robotdetection-"+version+r"\train\images"
train_label_path = r"..\robotdetection-"+version+r"\train\labels"
valid_image_path = r"..\robotdetection-"+version+r"\valid\images"
valid_label_path = r"..\robotdetection-"+version+r"\valid\labels"

train_files = os.listdir(train_image_path)
train_size = len(train_files)
valid_files = os.listdir(valid_image_path)
valid_size = len(valid_files)

count = 0
start = time.time()
for i, file in enumerate(train_files):
    # 从摄像头读取一帧图片
    image = cv.imread(train_image_path + "\\" + file)

    color_frame = image

    # 将RGB图像数据转换为NumPy数组
    color_image = np.asanyarray(color_frame)
    im0  = color_image
    # 图片预处理
    im = letterbox(im0)[0]  # padded resize
    im = im.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    im = np.ascontiguousarray(im)  # contiguous
    # 预测
    im = torch.from_numpy(im).to(model.device)
    im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # expand for batch dim
    pred = model(im)
    pred = non_max_suppression(pred, 0.25, 0.45, None, False, max_det=1000)
    count = count + 1
end  = time.time()
    
print("inference time:{:.2f} ms".format((end-start)/count*1000))

YOLOv5  2023-10-8 Python-3.10.11 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs


C:\Users\29777\Desktop\paper\object_detection\code\car_follow_diy\yolov5
inference time:12.19 ms
